Homework 02-02-experiment-tracking  

# Imports

In [1]:
!python -V

Python 3.9.19


In [2]:
import pandas as pd

In [3]:
import pickle

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error, root_mean_squared_error

In [6]:
import mlflow

mlflow.set_tracking_uri('sqlite:///mlflow.db')
mlflow.set_experiment('nyc-taxi-experiment')

<Experiment: artifact_location='/home/aleksandr/GitHub/MLOps_zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1716107757971, experiment_id='1', last_update_time=1716107757971, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

# Constant

In [7]:
PATH_TRAIN = '../01-intro/data/yellow_tripdata_2023-01.parquet'
PATH_VALID = '../01-intro/data/yellow_tripdata_2023-02.parquet'

# Preproc

In [8]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

# Load data

In [9]:
df_train = read_dataframe(PATH_TRAIN)
df_val = read_dataframe(PATH_VALID)

In [10]:
len(df_train), len(df_val)

(3009173, 2855951)

In [11]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [12]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [13]:
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((3009173, 516), (2855951, 516), (3009173,), (2855951,))

# Delete initial df for increasing  RAM volume

In [14]:
del df_train
del df_val

import gc
gc.collect()

0

In [15]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

# Use the direct RMSE function
rmse = root_mean_squared_error(y_val, y_pred)
rmse

7.81863017569759

# Save model

In [16]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

# Start experiment

In [17]:
with mlflow.start_run():

    mlflow.set_tag("developer", "aleks-sokolov")

    mlflow.log_param("train-data-path", PATH_TRAIN)
    mlflow.log_param("valid-data-path", PATH_VALID)

    alpha = 0.5
    mlflow.log_param("alpha", alpha)

    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)

    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

# Imports Hyperopt and create DMatrix 

In [18]:
import xgboost as xgb

In [19]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [20]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

# Delete initial df for increasing  RAM volume

In [23]:
del X_train
del X_val

import gc
gc.collect()

686

# Start MLFlow experiment

In [21]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [22]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:squarederror',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=5,
    trials=Trials()
)

  0%|          | 0/5 [00:00<?, ?trial/s, best loss=?]

/home/aleksandr/anaconda3/envs/exp-track-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [12:02:23] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:6.32549                          
[1]	validation-rmse:5.28021                          
[2]	validation-rmse:5.01031                          
[3]	validation-rmse:4.92829                          
[4]	validation-rmse:4.89095                          
[5]	validation-rmse:4.86560                          
[6]	validation-rmse:4.84996                          
[7]	validation-rmse:4.83572                          
[8]	validation-rmse:4.82042                          
[9]	validation-rmse:4.81133                          
[10]	validation-rmse:4.80095                         
[11]	validation-rmse:4.78784                         
[12]	validation-rmse:4.78027                         
[13]	validation-rmse:4.77427                         
[14]	validation-rmse:4.76797                         
[15]	validation-rmse:4.75996                         
[16]	validation-rmse:4.75653                         
[17]	validation-rmse:4.74991                         
[18]	validation-rmse:4.74450

/home/aleksandr/anaconda3/envs/exp-track-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [12:06:58] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:5.82861                                                    
[1]	validation-rmse:4.89840                                                    
[2]	validation-rmse:4.69688                                                    
[3]	validation-rmse:4.65996                                                    
[4]	validation-rmse:4.65278                                                    
[5]	validation-rmse:4.65158                                                    
[6]	validation-rmse:4.64780                                                    
[7]	validation-rmse:4.64746                                                    
[8]	validation-rmse:4.64726                                                    
[9]	validation-rmse:4.64762                                                    
[10]	validation-rmse:4.64740                                                   
[11]	validation-rmse:4.64830                                                   
[12]	validation-rmse:4.64974            

...

[0]	validation-rmse:7.00183                                                    
[1]	validation-rmse:5.64374                                                    
[2]	validation-rmse:5.09452                                                    
[3]	validation-rmse:4.87301                                                    
[4]	validation-rmse:4.77029                                                    
[5]	validation-rmse:4.72595                                                    
[6]	validation-rmse:4.70172                                                    
[7]	validation-rmse:4.68504                                                    
[8]	validation-rmse:4.67575                                                    
[9]	validation-rmse:4.66600                                                    
[10]	validation-rmse:4.66140                                                   
[11]	validation-rmse:4.65599                                                   
[12]	validation-rmse:4.64852            

/home/aleksandr/anaconda3/envs/exp-track-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [12:19:47] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:8.31037                                                   
[1]	validation-rmse:7.06749                                                   
[2]	validation-rmse:6.20116                                                   
[3]	validation-rmse:5.62226                                                   
[4]	validation-rmse:5.24825                                                   
[5]	validation-rmse:5.00440                                                   
[6]	validation-rmse:4.85299                                                   
[7]	validation-rmse:4.75725                                                   
[8]	validation-rmse:4.69468                                                   
[9]	validation-rmse:4.65329                                                   
[10]	validation-rmse:4.62707                                                  
[11]	validation-rmse:4.60963                                                  
[12]	validation-rmse:4.59790                        

/home/aleksandr/anaconda3/envs/exp-track-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [12:54:37] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:9.09409                                                    
[1]	validation-rmse:8.26742                                                    
[2]	validation-rmse:7.57593                                                    
[3]	validation-rmse:7.00461                                                    
[4]	validation-rmse:6.53017                                                    
[5]	validation-rmse:6.14492                                                    
[6]	validation-rmse:5.82755                                                    
[7]	validation-rmse:5.57215                                                    
[8]	validation-rmse:5.37024                                                    
[9]	validation-rmse:5.20984                                                    
[10]	validation-rmse:5.08251                                                   
[11]	validation-rmse:4.97905                                                   
[12]	validation-rmse:4.90008            

In [ ]:
mlflow.xgboost.autolog(disable=True)